In [1]:
%load_ext autoreload
%autoreload 2
import glob
import time, base64,ssl, os, re,json5, pprint,random , math, hashlib, inspect,requests
from selenium import webdriver# Controls the browser
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from app.utils import Helper
from app.operation_executor import OperationExecutor
import pandas as pd
import numpy as np
from io import StringIO
import undetected_chromedriver as uc

In [ ]:
import requests
import pandas as pd
from io import BytesIO

session = requests.Session()
session.headers.update({"User-Agent": "Mozilla/5.0"})

def fetch_rba():
    url = "https://www.rba.gov.au/statistics/tables/xls/f01d.xlsx"
    r = session.get(url, timeout=15)
    r.raise_for_status()
    return pd.read_excel(BytesIO(r.content))

def fetch_banxico():
    url = "https://www.banxico.org.mx/SieInternet/consultarDirectorioInternetAction.do?accion=consultarCuadro&idCuadro=CF101&sector=18&locale=en"
    r = session.get(url, timeout=15)
    r.raise_for_status()
    return pd.read_html(r.text)[0]

def fetch_bcra():
    url = "https://www.bcra.gob.ar/PublicacionesEstadisticas/Principales_variables_i.asp"
    r = session.get(url, timeout=15, verify=False)
    r.raise_for_status()
    return pd.read_html(r.text)[0]

def fetch_boc():
    url = "https://www.bankofcanada.ca/rates/interest-rates/canadian-interest-rates/"
    params = {
        "lookupPage": "lookup_canadian_interest.php",
        "rangeType": "dates",
        "ByDate_frequency": "daily",
        "dFrom": "2025-09-18",
        "dTo": "2025-09-19",
        "series[]": ["V39078", "V80691310"],
        "submit_button": "Submit"
    }
    r = session.get(url, params=params, timeout=15)
    r.raise_for_status()
    return pd.read_html(r.text)[0]

# --- Run and consolidate ---
with pd.ExcelWriter("central_banks.xlsx") as writer:
    fetch_rba().to_excel(writer, sheet_name="RBA", index=False)
    fetch_banxico().to_excel(writer, sheet_name="Banxico", index=False)
    fetch_bcra().to_excel(writer, sheet_name="BCRA", index=False)
    fetch_boc().to_excel(writer, sheet_name="BankOfCanada", index=False)

In [2]:
driver = webdriver.Chrome()
path = r"https://www.banxico.org.mx/SieInternet/consultarDirectorioInternetAction.do?accion=consultarCuadro&idCuadro=CF101&sector=18&locale=en"
driver.get(path)
wait = WebDriverWait(driver, 10)

wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
table = driver.find_element(By.CSS_SELECTOR, "#contenidoPrincipal")

html = table.get_attribute("outerHTML")
with open("mexico.html","w") as f:
    f.write(html)
time.sleep(3)
driver.quit()

In [4]:
import requests

url = r"https://www.bankofcanada.ca/rates/interest-rates/canadian-interest-rates/?lookupPage=lookup_canadian_interest.php&startRange=2015-09-18&rangeType=dates&dFrom=2025-09-16&dTo=2025-09-17&rangeValue=1&rangeWeeklyValue=1&rangeMonthlyValue=1&series%5B%5D=LOOKUPS_V39079&series%5B%5D=CL.CDN.MOST.1DL&series%5B%5D=V39078&series%5B%5D=V80691310&series%5B%5D=V122530&ByDate_frequency=daily&submit_button=Submit"

response = requests.get(url)

if response.status_code == "200":
    print(response.content)
    
print(response.status_code)

200


LAWER

In [ ]:


iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
driver.switch_to.frame(iframe)

dropdown_1 = ['Open Ended', 'Close Ended']
dropdown_2 = ['Equity', 'Debt', 'Hybrid', 'Solution Oriented', 'Other']

def select_dropdown(dropdown_element, value):
    arrow = dropdown_element.find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
    arrow.click()
    time.sleep(0.5)
    options = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))
    for opt in options:
        if opt.text.strip() == value:
            opt.click()
            break
    time.sleep(0.5)

# Only 3 options printed for nowwww
for op1 in dropdown_1:
    dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
    select_dropdown(dropdowns[0], op1)

    for op2 in dropdown_2:
        dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
        select_dropdown(dropdowns[1], op2)

        dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
        arrow = dropdowns[2].find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
        arrow.click()
        time.sleep(0.5)

        options = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))
        op3_options = [opt.text.strip() for opt in options]

        arrow.click()

        print(f"{op1} + {op2} → {op3_options}")

driver.quit()


In [ ]:
dropdown_1 = ['Open Ended', 'Close Ended']
op3_map = {
    "Equity": ['Large Cap', 'Large & Mid Cap', 'Flexi Cap', 'Multi Cap', 'Mid Cap',
               'Small Cap', 'Value', 'ELSS', 'Contra', 'Dividend Yield', 'Focused', 'Sectoral / Thematic'],
    "Debt": ['Long Duration', 'Medium to Long Duration', 'Short Duration', 'Medium Duration',
             'Money Market', 'Low Duration', 'Ultra Short Duration', 'Liquid', 'Overnight',
             'Dynamic Bond', 'Corporate Bond', 'Credit Risk', 'Banking and PSU', 'Floater',
             'FMP', 'Gilt', 'Gilt with 10 year constant duration'],
    "Hybrid": ['Aggressive Hybrid', 'Conservative Hyrbid', 'Equity Savings', 'Arbitrage',
               'Multi Asset Allocation', 'Dynamic Asset Allocation or Balanced Advantage', 'Balanced Hybrid'],
    "Solution Oriented": ["Children's", 'Retirement'],
    "Other": ['Index Funds ETFs', 'FoFs (Overseas/Domestic)']
}

dropdown_4_value = "All"

all_combos = [{ "op1": op1,"op2": op2,"op3": op3,"op4": dropdown_4_value} for op1 in dropdown_1 for op2, op3_list in op3_map.items() for op3 in op3_list]

# print(f"Total combinations: {len(all_combos)}")
# all_combos

In [13]:
from datetime import datetime, timedelta

def get_date_range(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    delta = (end - start).days + 1
    return [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(delta)]

# Example usage
dates = get_date_range("2019-04-01", "2025-03-31")
print(dates)


['2019-04-01', '2019-04-02', '2019-04-03', '2019-04-04', '2019-04-05', '2019-04-06', '2019-04-07', '2019-04-08', '2019-04-09', '2019-04-10', '2019-04-11', '2019-04-12', '2019-04-13', '2019-04-14', '2019-04-15', '2019-04-16', '2019-04-17', '2019-04-18', '2019-04-19', '2019-04-20', '2019-04-21', '2019-04-22', '2019-04-23', '2019-04-24', '2019-04-25', '2019-04-26', '2019-04-27', '2019-04-28', '2019-04-29', '2019-04-30', '2019-05-01', '2019-05-02', '2019-05-03', '2019-05-04', '2019-05-05', '2019-05-06', '2019-05-07', '2019-05-08', '2019-05-09', '2019-05-10', '2019-05-11', '2019-05-12', '2019-05-13', '2019-05-14', '2019-05-15', '2019-05-16', '2019-05-17', '2019-05-18', '2019-05-19', '2019-05-20', '2019-05-21', '2019-05-22', '2019-05-23', '2019-05-24', '2019-05-25', '2019-05-26', '2019-05-27', '2019-05-28', '2019-05-29', '2019-05-30', '2019-05-31', '2019-06-01', '2019-06-02', '2019-06-03', '2019-06-04', '2019-06-05', '2019-06-06', '2019-06-07', '2019-06-08', '2019-06-09', '2019-06-10', '2019

In [20]:
path = "RIL.COMPARE.xlsx"
import pandas as pd

df1 = pd.read_excel(path, sheet_name="Sheet1")
df2 = pd.read_excel(path, sheet_name="Sheet2")

df1.columns = ["ticker","date","time"] +["Ltp","Buy","BuyOty","Sell","SellQty","Ltq"]
df2.columns = ["time", "Ltp","Ltq","Buy","Sell"]

In [24]:
path = "RIL.COMPARE.xlsx"
import pandas as pd

df1 = pd.read_excel(path, sheet_name="Sheet1")
df2 = pd.read_excel(path, sheet_name="Sheet2")

df1.columns = ["ticker","date","time"] +["Ltp","Buy","BuyOty","Sell","SellQty","Ltq"]
df2.columns = ["time", "Ltp","Ltq","Buy","Sell"]
merged_df = pd.merge(df1, df2, on='time', how='outer', suffixes=('_df1', '_df2'))
merged_df.sort_values(by='time', inplace=True)

merged_df['Ltp_match']  = merged_df['Ltp_df1'] == merged_df['Ltp_df2']
merged_df['Ltq_match']  = merged_df['Ltq_df1'] == merged_df['Ltq_df2']
merged_df['Buy_match']  = merged_df['Buy_df1'] == merged_df['Buy_df2']
merged_df['Sell_match'] = merged_df['Sell_df1'] == merged_df['Sell_df2']

merged_df.to_excel("merged_ril.xlsx")

In [25]:
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

wb = load_workbook("merged_ril.xlsx")
ws = wb.active

green_fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")


match_cols = ['Ltp_match', 'Ltq_match', 'Buy_match', 'Sell_match']
match_indices = [merged_df.columns.get_loc(col) + 1 for col in match_cols]  # Excel is 1-indexed

for row in range(2, ws.max_row + 1):  # Skip header
    for col_idx in match_indices:
        cell = ws.cell(row=row, column=col_idx)
        if cell.value is True:
            cell.fill = green_fill

# Save the updated workbook
wb.save("merged_ril_validated.xlsx")

In [6]:
import os, time, json, shutil, glob
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
import pandas as pd

DATE_STR = "30-Jul-2025"
DOWNLOAD_DIR = os.path.join(os.getcwd(), f"mf_downloads_{DATE_STR}")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)

driver.get("https://www.amfiindia.com/otherdata/fund-performance")
iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
driver.switch_to.frame(iframe)

combos = all_combos

def select_dropdown(index, value, retries=3):
    for attempt in range(retries):
        try:
            dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
            dropdown = dropdowns[index]
            arrow = dropdown.find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
            arrow.click()
            time.sleep(0.5)

            options = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))
            for opt in options:
                if opt.text.strip() == value:
                    driver.execute_script("arguments[0].scrollIntoView(true);", opt)
                    opt.click()
                    time.sleep(0.5)
                    return
            raise ValueError(f"Option '{value}' not found")
        except Exception as e:
            print(f"Retrying select for '{value}' ({attempt+1}/{retries}) due to {e.__class__.__name__}")
            time.sleep(1)
    raise Exception(f"Failed to select '{value}' after retries")

def set_date(date_str):
    date_input = driver.find_element(By.CSS_SELECTOR, "input[bsdatepicker]")
    driver.execute_script("""
        arguments[0].value = arguments[1];
        arguments[0].dispatchEvent(new Event('input', { bubbles: true }));
        arguments[0].dispatchEvent(new Event('change', { bubbles: true }));
    """, date_input, date_str)
    time.sleep(1)

def click_go():
    go_button = driver.find_element(By.CSS_SELECTOR, "button[mat-raised-button]")
    go_button.click()
    time.sleep(2)

def download_excel():
    excel_icon = driver.find_element(By.CSS_SELECTOR, ".excel-border.cursor-pointer")
    excel_icon.click()
    time.sleep(1)

def if_data_avalaible():
    try:
        snackbars = driver.find_elements(By.CSS_SELECTOR, "snack-bar-container")
        for snackbar in snackbars:
            if "Data Not Available" in snackbar.text:
                close_icon = snackbar.find_element(By.XPATH, ".//mat-icon[contains(text(), 'close')]")
                close_icon.click()
                time.sleep(1)
                print("Data Not Available. Snackbar closed.")
                return False
    except:
        pass
    return True

# def modify_excel_data(combo):
#     time.sleep(2)
#     files = glob.glob(os.path.join(DOWNLOAD_DIR, "*.xlsx"))
#     latest_file = max(files, key=os.path.getctime)

#     original_name = os.path.basename(latest_file).replace(".xlsx", "")
#     combo_str = "_".join([combo["op1"], combo["op2"], combo["op3"], combo["op4"]]).replace(" ", "-")
#     new_name = f"Fund-Performance-{DATE_STR}-{combo_str}.xlsx"
#     new_path = os.path.join(DOWNLOAD_DIR, new_name)

#     counter = 1
#     while os.path.exists(new_path):
#         new_name = f"{original_name}_{combo_str}({counter}).xlsx"
#         new_path = os.path.join(DOWNLOAD_DIR, new_name)
#         counter += 1

#     os.rename(latest_file, new_path)

#     df_full = pd.read_excel(new_path, header=None)
#     df_top = df_full.iloc[:4]
#     df_data = df_full.iloc[4:].copy()

#     for idx, val in enumerate(combo.values()):
#         df_data.insert(loc=idx, column=f'Option_{idx+1}', value=val)

#     for idx in range(len(combo)):
#         df_top.insert(loc=idx, column=f'Option_{idx+1}', value=[""] * len(df_top))

#     df_final = pd.concat([df_top, df_data], ignore_index=True)

#     mod_dir = os.path.join(DOWNLOAD_DIR, "mod")
#     os.makedirs(mod_dir, exist_ok=True)
#     mod_path = os.path.join(mod_dir, new_name)

#     df_final.to_excel(mod_path, index=False, header=False)
#     print(f"Modified file saved to: {mod_path}")

not_avalaible_combos = []

for combo in combos:
    try:
        op1, op2, op3, op4 = combo["op1"], combo["op2"], combo["op3"], combo["op4"]

        print(f"\nPreparing combo: {op1}, {op2}, {op3}, {op4}")
    
        set_date(DATE_STR)
        select_dropdown(0, op1)
        select_dropdown(1, op2)
        select_dropdown(2, op3)
        select_dropdown(3, op4)

        # click_go()
        # time.sleep(2)
        # if not if_data_avalaible():
        #     not_avalaible_combos.append(combos)
        #     continue
                
        # download_excel()
        # print("Download triggered — waiting for file...")
        # modify_excel_data(combo)
        # time.sleep(2)

    except Exception as e:
        print(f"[ERROR] {combo}: {e}")

driver.quit()

# with open("not_open.json","w+") as f:
#     json.dump(not_avalaible_combos,f)


Preparing combo: Open Ended, Equity, Large Cap, All
Retrying select for 'Open Ended' (1/3) due to ElementClickInterceptedException
Retrying select for 'Open Ended' (2/3) due to TimeoutException
Retrying select for 'Open Ended' (3/3) due to ElementClickInterceptedException
[ERROR] {'op1': 'Open Ended', 'op2': 'Equity', 'op3': 'Large Cap', 'op4': 'All'}: Failed to select 'Open Ended' after retries

Preparing combo: Open Ended, Equity, Large & Mid Cap, All
Retrying select for 'Open Ended' (1/3) due to ElementClickInterceptedException
Retrying select for 'Open Ended' (2/3) due to InvalidSessionIdException
Retrying select for 'Open Ended' (3/3) due to InvalidSessionIdException
[ERROR] {'op1': 'Open Ended', 'op2': 'Equity', 'op3': 'Large & Mid Cap', 'op4': 'All'}: Failed to select 'Open Ended' after retries

Preparing combo: Open Ended, Equity, Flexi Cap, All
[ERROR] {'op1': 'Open Ended', 'op2': 'Equity', 'op3': 'Flexi Cap', 'op4': 'All'}: Message: invalid session id; For documentation on t

In [ ]:
def create_grand_sheet(path:str):
    all_dataframes = []
    excel_files = sorted(glob.glob(os.path.join(path, "*.xlsx")))
    
    all_data_path = os.path.join(path, "ALL_DATA_FILE.xlsx")
    grand_path = os.path.join(path, f"Grand_Stacked_{DATE_STR}.xlsx")
    
    #stack
    for file in excel_files:
        df = pd.read_excel(file, header=None)
        empty_rows = pd.DataFrame([[""] * df.shape[1]] * 3)
        all_dataframes.append(df)
        all_dataframes.append(empty_rows)

    grand_df = pd.concat(all_dataframes, ignore_index=True)
    grand_df.to_excel(grand_path, index=False, header=False)
    
    #one shot
    with pd.ExcelWriter(all_data_path, engine='openpyxl') as writer:
        for i, file in enumerate(excel_files):
            sheet_name = f"Sheet{i+1}"
            df = pd.read_excel(file, header=None)
            df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

    print(f" `ALL_DATA_FILE` created with {len(excel_files)} sheets: {all_data_path}")
    print(f"Grand Excel created with {len(excel_files)} : {grand_path}")


path = r"C:\Users\kaustubh.keny\Projects\office-work\selenium-scrape\mf_downloads_30-Jul-2025\mod"

create_grand_sheet(path)


In [ ]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
from urllib.parse import urljoin
import os
ops = OperationExecutor()
path = r"25-09-10T16-42-02_cache.json"
# path = r"25-09-08T18-33-09_cache.json5"

    
root_file = os.path.basename(path).split("_")[0]
function_to_execute = {
"primary":[["normalize_df","value","norm_table","table_html"],["sha1","value","SHA_ONE","pdf"]],
"secondary":[["sha1","norm_table","SHA_ONE"]],
}
try:
    data = Helper.load_json(path, typ="json5")
    processed_data = ops.runner(data,function_to_execute)
    Helper.save_json(processed_data,f"{root_file}_process.json")
except Exception as e:
    print(f"\nError in Processing.. Skipping, {e}")


In [ ]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
ops = OperationExecutor()
yesterday = Helper.load_json("25-09-09T15-14-38_process.json")
today = Helper.load_json("25-09-10T16-42-02_process.json")

result = ops.process_comparison(yesterday,today,key="SHA_ONE")
Helper.save_json(result, "compare-080925.json")

ops.generate_sorted_excel_report(result,output_path="Despoit_Rate_Report.xlsx")

In [ ]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
ops = OperationExecutor()

path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\1109_1009_compare\compare-080925.json"
result = Helper.load_json(path)
ops.generate_sorted_excel_report(result,output_path="Despoit_Rate_Report.xlsx")

'Despoit_Rate_Report.xlsx'

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()
driver.get("https://itat.gov.in/judicial/tribunalorders")

bench_dropdown = Select(driver.find_element(By.ID, "bench_name_2"))
bench_dropdown.select_by_index(1)

appeal_dropdown = Select(driver.find_element(By.ID, "app_type_2"))
appeal_dropdown.select_by_index(1)

date_input = driver.find_element(By.ID, "order_date")
driver.execute_script("arguments[0].removeAttribute('readonly')", date_input)
date_input.clear()
date_input.send_keys("11-09-2025")
time.sleep(5.2)

search_btn = driver.find_element(By.ID, "b2")
search_btn.click()
